## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,54.90,50,86,4.00,overcast clouds
1,1,Tebingtinggi,ID,3.3285,99.1625,87.85,52,91,4.63,overcast clouds
2,2,Kodiak,US,57.7900,-152.4072,62.11,39,1,10.36,clear sky
3,3,Carupano,VE,10.6678,-63.2585,75.60,90,3,8.28,clear sky
4,4,Hermanus,ZA,-34.4187,19.2345,57.90,63,87,6.24,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                        (city_data_df["Max Temp"] >= min_temp)]
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Tebingtinggi,ID,3.3285,99.1625,87.85,52,91,4.63,overcast clouds
3,3,Carupano,VE,10.6678,-63.2585,75.60,90,3,8.28,clear sky
7,7,Kahului,US,20.8947,-156.4700,78.82,81,40,10.36,scattered clouds
8,8,Avarua,CK,-21.2078,-159.7750,80.65,83,20,4.61,light rain
12,12,Jinchang,CN,38.4953,102.1739,76.15,11,0,17.16,clear sky


In [13]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
######          No empty row        ######

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Tebingtinggi,ID,87.85,overcast clouds,3.3285,99.1625,
3,Carupano,VE,75.60,clear sky,10.6678,-63.2585,
7,Kahului,US,78.82,scattered clouds,20.8947,-156.4700,
8,Avarua,CK,80.65,light rain,-21.2078,-159.7750,
12,Jinchang,CN,76.15,clear sky,38.4953,102.1739,
19,Kapaa,US,80.58,scattered clouds,22.0752,-159.3190,
32,Butaritari,KI,82.08,overcast clouds,3.0707,172.7902,
36,Kendari,ID,86.05,broken clouds,-3.9450,122.4989,
39,Dong Hoi,VN,85.41,overcast clouds,17.4833,106.6000,
40,Omboue,GA,77.49,broken clouds,-1.5746,9.2618,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Tebingtinggi,ID,87.85,overcast clouds,3.3285,99.1625,Hotel Malibou tebing tinggi
3,Carupano,VE,75.60,clear sky,10.6678,-63.2585,Hotel La dorada
7,Kahului,US,78.82,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
8,Avarua,CK,80.65,light rain,-21.2078,-159.7750,Paradise Inn
12,Jinchang,CN,76.15,clear sky,38.4953,102.1739,Mingshihui Jinchang International Hotel


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df = hotel_df
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))